## Load Libs

In [825]:
import pandas as pd
import os
import sys
import csv
import data_validation_helper as dvh
import data_transform_helper as dth
import database_helper as dh
import importlib
importlib.reload(dvh)
importlib.reload(dfh)

<module 'data_transform_helper' from 'C:\\Users\\a30006212\\Downloads\\AGL Study\\Study\\Shell Energy Data Engineering Test\\Output\\3.Transfrom Step Implementation\\data_transform_helper.py'>

## Data Transform

In [826]:
nmi_path = '..\\1.Input Data Inspection\\Data\\nmi_info.csv'
consumption_folder = '..\\1.Input Data Inspection\\Data\\ConsumptionData'

In [827]:
output_path='Transformed\\'
nmi_df = dth.transform_nmi_master(nmi_path, output_path)
nmi_df.head()

,NMI,STATE,INTERVAL
0,NMIM1,QLD,30
1,NMIS2,NSW,30
2,NMIS3,NSW,30
3,NMIA2,VIC,15
4,NMIA1,VIC,15


In [828]:
consumption_df = dth.transform_consumption(consumption_folder)
consumption_df.head()

,TRANSFORMED_AESTTIME,AESTTIME,QUANTITY,UNIT,STATE,DATE,YEAR,YEARDAY,MONTH,MONTHNAME,...,MINUTE,WEEKDAY,WEEKEND,TIME,MONTHDAY,HOURMINUTE,SESSION,SEASON,NMI,OUTLIER
0,2017-10-01 00:00:00,2017-10-01 00:00:00,7.99,KWH,VIC,2017-10-01,2017,274,10,October,...,0,6,1,00:00:00,10-01,00:00,1,4,NMIA1,0
1,2017-10-01 00:15:00,2017-10-01 00:15:00,7.00,KWH,VIC,2017-10-01,2017,274,10,October,...,15,6,1,00:15:00,10-01,00:15,1,4,NMIA1,0
2,2017-10-01 00:30:00,2017-10-01 00:30:00,8.20,KWH,VIC,2017-10-01,2017,274,10,October,...,30,6,1,00:30:00,10-01,00:30,1,4,NMIA1,0
3,2017-10-01 00:45:00,2017-10-01 00:45:00,8.01,KWH,VIC,2017-10-01,2017,274,10,October,...,45,6,1,00:45:00,10-01,00:45,1,4,NMIA1,0
4,2017-10-01 01:00:00,2017-10-01 01:00:00,9.01,KWH,VIC,2017-10-01,2017,274,10,October,...,0,6,1,01:00:00,10-01,01:00,1,4,NMIA1,0


In [829]:
default_timezone = 'Australia/Brisbane'
transform_dict = {'VIC': 'Australia/Victoria',
                  'NSW': 'Australia/NSW',
                  'SA': 'Australia/Adelaide',
                   'WA': 'Australia/West',
                  'TAS':'Australia/Tasmania',
                   'QLD': 'Australia/Queensland',
                   'ACT': 'Australia/ACT'}

In [847]:
subset_consumption_df = consumption_df[consumption_df['STATE'] == 'NSW'][['TRANSFORMED_AESTTIME', 'AESTTIME']]
subset_consumption_df

,TRANSFORMED_AESTTIME,AESTTIME
0,2017-10-01 00:00:00,2017-10-01 00:00:00
1,2017-10-01 00:30:00,2017-10-01 00:30:00
2,2017-10-01 01:00:00,2017-10-01 01:00:00
3,2017-10-01 01:30:00,2017-10-01 01:30:00
4,2017-10-01 03:00:00,2017-10-01 02:00:00
...,...,...
17515,2018-09-30 21:30:00,2018-09-30 21:30:00
17516,2018-09-30 22:00:00,2018-09-30 22:00:00
17517,2018-09-30 22:30:00,2018-09-30 22:30:00
17518,2018-09-30 23:00:00,2018-09-30 23:00:00


In [852]:
subset_consumption_df = consumption_df[consumption_df['STATE'] == 'NSW'][['TRANSFORMED_AESTTIME', 'AESTTIME']]
subset_consumption_df['ACTUAL'] = subset_consumption_df['AESTTIME'].dt.tz_localize(default_timezone).dt.tz_convert(transform_dict['NSW'])
subset_consumption_df = subset_consumption_df.set_index(subset_consumption_df['ACTUAL'])
subset_consumption_df['ACTUAL'] = subset_consumption_df.index.strftime('%Y-%m-%d %H:%M:%S')
subset_consumption_df = subset_consumption_df.astype({'ACTUAL':'datetime64[ns]'}).set_index('ACTUAL')
subset_consumption_df.reset_index(drop=False, inplace=True)
result = subset_consumption_df['TRANSFORMED_AESTTIME'] == subset_consumption_df['ACTUAL']
list(result.unique())[0]

True

In [843]:
nmi_df.dtypes.to_list() == ['O', 'O', 'int32']

True

In [749]:
consumption_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223672 entries, 0 to 17519
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   TRANSFORMED_AESTTIME  223672 non-null  datetime64[ns]
 1   AESTTIME              223672 non-null  datetime64[ns]
 2   QUANTITY              223672 non-null  float64       
 3   UNIT                  223672 non-null  object        
 4   STATE                 223672 non-null  object        
 5   DATE                  223672 non-null  object        
 6   YEAR                  223672 non-null  int64         
 7   YEARDAY               223672 non-null  int64         
 8   MONTH                 223672 non-null  int64         
 9   MONTHNAME             223672 non-null  object        
 10  WEEK                  223672 non-null  UInt32        
 11  DAY                   223672 non-null  int64         
 12  DAYNAME               223672 non-null  object        
 13  

## Setup Database

In [736]:
df = pd.read_csv('..\\1.Input Data Inspection\\Data\\ConsumptionData\\NMIA1.csv')
df.head()

,AESTTime,Quantity,Unit
0,2017-10-01 00:00:00,7.99,kWh
1,2017-10-01 00:15:00,7.00,kWh
2,2017-10-01 00:30:00,8.20,kWh
3,2017-10-01 00:45:00,8.01,kWh
4,2017-10-01 01:00:00,9.01,kWh


This will use the helper functions defined in the database_helper module

In [750]:
# if not exist create a db file
if not os.path.exists('Transformed\\pythonsqlite.db'):
    with open('Transformed\\pythonsqlite.db', 'w'): pass
database = r'Transformed\\pythonsqlite.db'

In [752]:
# create a database connection
conn = dh.create_connection(database)

nmi_df.to_sql(name='nmi', con=conn, if_exists='replace', index=False)
consumption_df.to_sql(name='consumption', con=conn, if_exists='replace', index=False)

### Table Check

In [751]:
nmi_master = 'Transformed\\transformed_nmi_info.csv'
consumption_data = 'Transformed\\ConsumptionData\\transformed_consumption_data_merged.csv'

In [753]:
# Check total nmi rows in raw data
len(nmi_df)

12

In [754]:
conn = dh.create_connection(database)
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM nmi")
row = cur.fetchall()
print(row[0][0]) 

12


In [755]:
# Check total payment transaction rows in raw data
len(consumption_df)

223672

In [756]:
conn = dh.create_connection(database)
cur = conn.cursor()
cur.execute("SELECT COUNT(*) FROM consumption")
row = cur.fetchall()
print(row[0][0])

223672


## Generate Report

In [771]:
sql_daily_hours_consumption = """
SELECT 
NMI, 
YEAR, 
MONTH, 
DAY,
HOUR,
STATE,
CASE
WHEN SESSION = 1 THEN 'Late Night'
WHEN SESSION = 2 THEN 'Early Morning'
WHEN SESSION = 3 THEN 'Morning'
WHEN SESSION = 4 THEN 'Noon'
WHEN SESSION = 5 THEN 'Eve'
ELSE 'Night' END AS SESSION,

CASE 
WHEN SEASON = 1 THEN 'Spring'
WHEN SEASON = 2 THEN 'Summer'
WHEN SEASON = 3 THEN 'Autumn'
ELSE 'Winter' END AS SEASON,

SUM(QUANTITY) AS TOTAL_CONSUMPTION, 

FROM consumption
GROUP BY 
NMI,
YEAR, 
MONTH, 
DAY,
HOUR,
STATE,
SESSION,
SEASON
ORDER BY NMI
"""

### Result Check

In [772]:
conn = dh.create_connection(database)
report_df = pd.read_sql_query(sql_daily_hours_consumption, conn)
report_df

,NMI,YEAR,MONTH,DAY,HOUR,STATE,SESSION,SEASON,TOTAL_CONSUMPTION
0,NMIA1,2017,10,1,0,VIC,Late Night,Winter,31.20
1,NMIA1,2017,10,1,1,VIC,Late Night,Winter,33.65
2,NMIA1,2017,10,1,3,VIC,Late Night,Winter,33.98
3,NMIA1,2017,10,1,4,VIC,Late Night,Winter,39.44
4,NMIA1,2017,10,1,5,VIC,Early Morning,Winter,48.32
...,...,...,...,...,...,...,...,...,...
94304,NMIS3,2018,9,30,19,NSW,Eve,Winter,27.68
94305,NMIS3,2018,9,30,20,NSW,Eve,Winter,27.36
94306,NMIS3,2018,9,30,21,NSW,Night,Winter,27.36
94307,NMIS3,2018,9,30,22,NSW,Night,Winter,28.00


The report takes the form of a table, with the following columns:
1. NMI - 
2. 

In [774]:
report_df.to_csv('../4.Analysis/NMI_Hourly_Consumption_Report.csv',index=False)